In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix,  roc_auc_score
import numpy as np
import time
"Own made libraries"
import database as db
import scrapping.scrapperSelenium as scrapperSelenium
import scrapping.scrapperSoup as scrapperSoup
import textCleaning as tc

[nltk_data] Downloading package wordnet to C:\Users\Roy
[nltk_data]     Dijkstra\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Roy
[nltk_data]     Dijkstra\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\Roy
[nltk_data]     Dijkstra\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
dfReviewsDb = db.retrieve_table_into_df('cleanedhoteldata')

In [3]:
X = dfReviewsDb["lemReviews"]
y = dfReviewsDb["label"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=69)

In [4]:
vect = CountVectorizer(max_features=1000)

X_train_vect = vect.fit_transform(X_train)
X_test_vect = vect.transform(X_test)

In [5]:
randomForest = RandomForestClassifier(n_estimators=2171, min_samples_split=2, min_samples_leaf=1, max_depth=200, criterion="gini", bootstrap=1, n_jobs=-1)

randomForest.fit(X_train_vect, y_train)

RandomForestClassifier(bootstrap=1, max_depth=200, n_estimators=2171, n_jobs=-1)

In [6]:
y_pred = randomForest.predict(X_test_vect)
y_prob = randomForest.predict_proba(X_test_vect)[::, 1]

In [7]:
print("Accuracy: {:.2f}%".format(accuracy_score(y_test, y_pred) * 100))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("AUC: \n", roc_auc_score(y_test, y_prob))

Accuracy: 92.98%
Confusion Matrix:
 [[ 1452  6593]
 [  649 94458]]
AUC: 
 0.8757931067440621


In [8]:
import pickle
filename = 'finalized_model_best_params.sav'
pickle.dump(randomForest, open(filename, 'wb'))

## Cleaning data from Hotel_Reviews.csv

In [2]:
start_time = time.time()

dfReviews = pd.read_csv("Hotel_Reviews.csv")
dfReviews = tc.kaggle_strip_reviews(dfReviews)
dfReviews = tc.kaggle_label_data(dfReviews)

E:\Roy Dijkstra\School\BigData\BDSE\Individual Assignment 1\textCleaning.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfStrippedReviews["Review"] = dfReviews["Negative_Review"].apply(lambda x: x.replace("No Negative", "")) + \


In [4]:
dfReviews = tc.clean_data(dfReviews)

end_time = time.time()

In [6]:
db.insert_df_into_db(dfReviews, "cleanedhoteldata", "append")

Inserting data into database per 10000 rows


In [7]:
time_lapsed = end_time - start_time
print(time_lapsed)

282.41862750053406


### Clean own data

In [ ]:
dfReviewsOwn = pd.read_csv("own_reviews.csv", sep=',')
dfReviewsOwn = tc.own_label_data(dfReviewsOwn)
dfReviewsOwn = tc.clean_data(dfReviewsOwn)

#### Insert own data into cleanedhoteldata database

In [ ]:
db.insert_df_into_db(dfReviewsOwn, "cleanedhoteldata", "append")